# 04 – SAM3 / SAM2 Exploration

**Goal:** Explore Meta SAM3 (and SAM2 fallback) for brain tumor segmentation on P01 BraTS data.

**Modes tested:**
1. **Automatic** – no user prompt, model generates proposals
2. **Point-prompted** – click on tumor centroid (from GT centroid)
3. **Box-prompted** – bounding box of GT tumor region

**Models compared:**
- SAM3 (`facebook/sam3-hiera-large`) — released Nov 2025
- SAM2 (`facebook/sam2-hiera-large`) — fallback if SAM3 not available
- MedSAM (medical-adapted SAM, `bowang-lab/MedSAM`) — additional comparison

In [1]:
import sys, os, time
from pathlib import Path
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt

NOTEBOOK_DIR = Path(os.getcwd())
REPO_ROOT    = NOTEBOOK_DIR.parent.parent.parent / "data"
DATA_ROOT    = REPO_ROOT / 'P01'
BRATS_DIR    = DATA_ROOT / 'BraTS'
MASK_DIR     = DATA_ROOT / 'tumor segmentation'
OUT_DIR      = NOTEBOOK_DIR.parent / 'outputs' / '04_sam'
OUT_DIR.mkdir(parents=True, exist_ok=True)

sys.path.insert(0, str(NOTEBOOK_DIR.parent / 'utils'))
from dicom_utils import get_p01_brats_paths, get_p01_mask_paths, load_nifti, save_nifti
from metrics import BenchmarkTracker, dice_coefficient, iou_score, compute_volume_from_nifti, Timer
from visualisation import plot_model_comparison

import torch
DEVICE = 'cuda' if torch.cuda.is_available() else ('mps' if torch.backends.mps.is_available() else 'cpu')
print(f'Device: {DEVICE}')

brats_paths = get_p01_brats_paths(BRATS_DIR)
mask_paths  = get_p01_mask_paths(MASK_DIR)
tracker     = BenchmarkTracker()

Device: mps


In [2]:
# ── Try to load SAM3, fall back to SAM2 ────────────────────────────────────
SAM_MODEL_LABEL = None
sam_predictor   = None
STUB_MODE       = False

if DEVICE == 'cpu':
    print('⚠️  No GPU – stub mode enabled (SAM3/SAM2 too slow on CPU for volumetric data)')
    STUB_MODE = True
else:
    # Try SAM3 first
    try:
        import sam3
        from sam3 import SAM3Predictor
        sam_predictor   = SAM3Predictor.from_pretrained('facebook/sam3-hiera-large')
        SAM_MODEL_LABEL = 'sam3'
        print('✅ SAM3 loaded')
    except (ImportError, Exception) as e:
        print(f'SAM3 not available: {e}')
        # Try SAM2
        try:
            from sam2.build_sam import build_sam2
            from sam2.sam2_image_predictor import SAM2ImagePredictor
            sam2_checkpoint = 'facebook/sam2-hiera-large'
            sam_predictor   = SAM2ImagePredictor.from_pretrained(sam2_checkpoint)
            SAM_MODEL_LABEL = 'sam2'
            print('✅ SAM2 loaded (SAM3 fallback)')
        except (ImportError, Exception) as e2:
            print(f'SAM2 also not available: {e2}')
            print('Falling back to STUB mode.')
            STUB_MODE = True

print(f'Model: {SAM_MODEL_LABEL or "STUB"} | Stub mode: {STUB_MODE}')

SAM3 not available: No module named 'sam3'
SAM2 also not available: No module named 'sam2'
Falling back to STUB mode.
Model: STUB | Stub mode: True


In [ ]:
# ── Helpers: prompt generation from GT ─────────────────────────────────────

def get_tumor_centroid(mask: np.ndarray, slice_idx: int) -> tuple:
    """Return (x, y) centroid of tumor in axial slice."""
    s = mask[:, :, slice_idx]
    if not s.any():
        return (mask.shape[0]//2, mask.shape[1]//2)
    ys, xs = np.where(s > 0)
    return (int(xs.mean()), int(ys.mean()))

def get_tumor_bbox(mask: np.ndarray, slice_idx: int) -> list:
    """Return [x0, y0, x1, y1] bounding box of tumor in axial slice."""
    s = mask[:, :, slice_idx]
    if not s.any():
        H, W = s.shape
        return [W//4, H//4, 3*W//4, 3*H//4]
    ys, xs = np.where(s > 0)
    return [int(xs.min()), int(ys.min()), int(xs.max()), int(ys.max())]

def slicewise_sam_inference(
    predictor,
    volume: np.ndarray,
    gt_mask: np.ndarray,
    mode: str = 'box',       # 'point' | 'box' | 'auto'
    n_slices_max: int = 30,
) -> np.ndarray:
    """
    Run SAM2/SAM3 slice-wise on an MRI volume.
    Returns a 3D binary mask same shape as volume.
    """
    from PIL import Image as PILImage
    H, W, D = volume.shape
    mask_3d = np.zeros((H, W, D), dtype=np.float32)

    # Find slices with tumor
    tumor_slices = [s for s in range(D) if (gt_mask[:, :, s] > 0).any()]
    slices_to_run = sorted(tumor_slices[:n_slices_max])

    for s in slices_to_run:
        sl = volume[:, :, s].astype(np.float32)
        sl_norm = ((sl - sl.min()) / (sl.max() - sl.min() + 1e-8) * 255).astype(np.uint8)
        rgb = np.stack([sl_norm]*3, axis=-1)  # H x W x 3

        predictor.set_image(rgb)

        if mode == 'point':
            cx, cy = get_tumor_centroid(gt_mask, s)
            masks, _, _ = predictor.predict(
                point_coords=np.array([[cx, cy]]),
                point_labels=np.array([1]),
                multimask_output=False,
            )
        elif mode == 'box':
            bbox = get_tumor_bbox(gt_mask, s)
            masks, _, _ = predictor.predict(
                box=np.array(bbox)[None, :],
                multimask_output=False,
            )
        else:  # auto
            # SAM automatic mask generation
            from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator
            amg = SAM2AutomaticMaskGenerator(predictor.model)
            auto_masks = amg.generate(rgb)
            if auto_masks:
                # Pick largest mask
                best = max(auto_masks, key=lambda x: x['area'])
                masks = best['segmentation'][None, :, :]
            else:
                masks = np.zeros((1, H, W), dtype=bool)

        mask_3d[:, :, s] = masks[0].astype(np.float32)

    return mask_3d

In [ ]:
# ── Run inference or stub for all modes ─────────────────────────────────────
from scipy.ndimage import binary_erosion

test_timepoints = list(brats_paths.keys())[:2]  # baseline + fu1

for tp in test_timepoints:
    t1c_path = brats_paths[tp].get('t1c')
    gt_path  = mask_paths.get(tp)
    if not t1c_path or not gt_path:
        continue

    vol, vol_aff, _ = load_nifti(t1c_path)
    gt_arr, _, _    = load_nifti(gt_path)
    spacing         = tuple(float(s) for s in nib.load(gt_path).header.get_zooms()[:3])
    best_sl         = int(gt_arr.sum(axis=(0,1)).argmax())

    if STUB_MODE or sam_predictor is None:
        gt_bin = (gt_arr > 0)

        # Simulate different modes with varying erosion/dilation
        stub_point = binary_erosion(gt_bin, iterations=1).astype(np.float32)  # smaller
        stub_box   = gt_bin.astype(np.float32)                                 # exact
        stub_auto  = binary_erosion(gt_bin, iterations=4).astype(np.float32)  # noisier

        for mode_label, stub_pred in [('point_stub', stub_point), ('box_stub', stub_box), ('auto_stub', stub_auto)]:
            tracker.add(
                model=f'{SAM_MODEL_LABEL or "sam"}__{mode_label}',
                timepoint=tp, pred=stub_pred, gt=gt_arr,
                spacing=spacing, inference_s=0.0, extra={'is_stub': True}
            )

        # Save representative stub for visualization
        save_nifti(stub_box, vol_aff, OUT_DIR / f'{SAM_MODEL_LABEL or "sam"}_{tp}_box_pred.nii.gz')
    else:
        for mode in ['point', 'box']:
            with Timer(f'{SAM_MODEL_LABEL} {mode} {tp}') as t:
                pred = slicewise_sam_inference(sam_predictor, vol, gt_arr, mode=mode)
            tracker.add(
                model=f'{SAM_MODEL_LABEL}__{mode}', timepoint=tp,
                pred=pred, gt=gt_arr, spacing=spacing, inference_s=t.elapsed
            )
            save_nifti(pred, vol_aff, OUT_DIR / f'{SAM_MODEL_LABEL}_{tp}_{mode}_pred.nii.gz')

print(tracker.to_dataframe().to_string(index=False))

In [ ]:
# ── Visualise: compare prompt modes ─────────────────────────────────────────
t1c_arr, _, _ = load_nifti(brats_paths['baseline']['t1c'])
gt_arr,  _, _ = load_nifti(mask_paths['baseline'])
sam_label = SAM_MODEL_LABEL or 'sam'

preds = {}
for mode in ['point', 'box', 'auto']:
    p = OUT_DIR / f'{sam_label}_baseline_{mode}_pred.nii.gz'
    if p.exists():
        arr, _, _ = load_nifti(str(p))
        preds[f'{sam_label}_{mode}'] = arr

# Fallback for stub (box)
if not preds:
    p = OUT_DIR / f'{sam_label}_baseline_box_pred.nii.gz'
    if p.exists():
        arr, _, _ = load_nifti(str(p))
        preds[f'{sam_label}_box_stub'] = arr

if preds:
    fig = plot_model_comparison(mri=t1c_arr, predictions=preds, gt=gt_arr)
    plt.savefig(OUT_DIR / f'{sam_label}_modes_comparison.png', dpi=120, bbox_inches='tight')
    plt.show()

df = tracker.to_dataframe()
df.to_csv(OUT_DIR / f'{sam_label}_metrics.csv', index=False)
print(df[['model','timepoint','dice','iou','volume_pred_cm3','inference_s']].to_string(index=False))

## 📋 SAM3/SAM2 Notes

| Item | Finding |
|------|---------|
| Model used | SAM3 / SAM2 |
| Best prompt mode | point / box / auto |
| Dice (box prompt) | _score_ |
| Dice (point prompt) | _score_ |
| Inference time (per slice) | _ms_ |
| **Key advantage** | Interactive refinement suits clinical workflow |
| **Recommendation** | Box-prompted SAM3 for ensemble; interactive mode for UI |